**British Airways Revenue Forecasting 3** <br>
Oxford Saïd Business School <br>
Analysis 3: Regressions <br><br>

*Jake Schneider* <br>
MBA Candidate <br>
November 2020 <br>

----

**Load Libraries**

In [1]:
#Import libraries
import sys
import pandas as pd
from datetime import date, datetime, time, timedelta
import pendulum
import json
import requests
import numpy as np
from numpy import NaN
from numpy import nanmean
import math

import collections

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style='ticks', context='talk')

from matplotlib.offsetbox import AnchoredText
from sklearn import linear_model
from sklearn.metrics import r2_score
import statsmodels.formula.api as sm
#import fbprophet

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import datapane as dp

import warnings
import itertools

----

**Load & Clean Macro & BA Regression Data Set**

In [2]:
# Load Continental Data

continental_df = pd.read_csv('../3_Outputs/Data/continental_df.csv')

In [3]:
# Load BA Data

ba_df = pd.read_excel('../2_Inputs/British Airways/BA Income Statement Max.xlsx', sheet_name = 'Input Data')

In [4]:
# Load BA Data 2

ba_df2 = pd.read_excel('../2_Inputs/British Airways/BA Income Statement Max.xlsx', sheet_name = 'Regional Input Data')

*Merge Macro Data with BA Income Statement*

In [5]:
# Merge Merge1 and IMF_DF

merge_df1 = pd.merge(continental_df, ba_df, how = 'left', left_on = ['Year'], right_on = ['Year'])
merge_df1

,region,Year,longitude,latitude,Population growth (annual %),"Population, total","Current account balance, Percent of GDP,","Employment, Persons, Millions","General government gross debt, Percent of GDP,","General government net debt, National currency, Billions",...,U.S. GAAP Summary,Net Income,NI to Common Incl. Extra Items,Diluted Net Income,Basic EPS Incl. Extra Items,Basic EPS Excl. Extra Items.1,Weighted Avg. Basic Shares Out..1,Diluted EPS Incl. Extra Items,Diluted EPS Excl. Extra Items.1,Weighted Avg. Diluted Shares Out..1
0,East Asia & Pacific,1980.0,102.080504,4.391124,1.947838,4.166191e+07,-2.671579,13.360000,48.810000,44619.100000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,East Asia & Pacific,1981.0,102.080504,4.391124,2.281081,4.229082e+07,-3.943158,13.531667,53.950000,57895.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,East Asia & Pacific,1982.0,102.080504,4.391124,2.442973,4.298174e+07,-3.261053,13.731667,58.960000,73262.900000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,East Asia & Pacific,1983.0,102.080504,4.391124,2.454054,4.367263e+07,-2.458421,13.896667,64.920000,90702.100000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,East Asia & Pacific,1984.0,102.080504,4.391124,2.448919,4.432978e+07,0.043158,13.988333,66.970000,51893.755000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,Combined Other Five Regions,2021.0,-14.380390,24.556544,NaN,5.300439e+07,-4.851083,23.916950,80.462533,338642.413089,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,Combined Other Five Regions,2022.0,-14.380390,24.556544,NaN,5.345879e+07,-4.076186,NaN,79.868244,417685.624486,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136,Combined Other Five Regions,2023.0,-14.380390,24.556544,NaN,5.391120e+07,-3.422181,NaN,78.958078,522116.235670,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137,Combined Other Five Regions,2024.0,-14.380390,24.556544,NaN,5.436135e+07,-2.997837,NaN,77.979511,661198.283142,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Merge Merge

reg_df = pd.merge(merge_df1, ba_df2, how = 'left', left_on = ['region', 'Year'], right_on = ['region', 'Year'])
#reg_df.tail(25)

In [7]:
# Create Log Variables

reg_df['log_GDP_per_capita'] = np.log(reg_df['Gross domestic product per capita, constant prices, Purchasing power parity; 2017 international dollar, Units'])
reg_df['log_Population_total'] = np.log(reg_df['Population, total'])
reg_df['log_Investment_total_perc_gdp'] = np.log(reg_df['Total investment, Percent of GDP, '])
reg_df['log_Net_Income'] = np.log(reg_df['  Net Income'])
reg_df['log_regional_revenues'] = np.log(reg_df['regional_revenues'])

reg_df['investment_total_perc_gdp'] = reg_df['Total investment, Percent of GDP, '] 

#reg_df.head(15)

In [8]:
# Create Region Dummy Variables

dummy_df = pd.get_dummies(reg_df['region'])
dummy_df

,Combined Other Five Regions,East Asia & Pacific,South Asia
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
137,1,0,0
138,1,0,0
139,1,0,0
140,1,0,0


In [9]:
# Concat DF

reg_df = pd.concat([reg_df, dummy_df], axis = 1)

In [10]:
# Rename 

reg_df = reg_df.rename(columns={"Combined Other Five Regions": "five_other_regions", 
                                "East Asia & Pacific": "east_asia", 
                                "South Asia": "south_asia" })

In [11]:
# Output Reg DF

reg_df = reg_df.to_csv('../3_Outputs/Data/reg_df.csv', index = False)

---

**Regressions**

In [12]:
# Load Continental Data

reg_final_df = pd.read_csv('../3_Outputs/Data/reg_df.csv')

*OLS*

In [13]:
# Regression 1

#normal_ols = sm.ols(formula='log_Net_Income ~ log_GDP_per_capita + log_Population_total + log_Investment_total_perc_gdp',
#                          data=reg_df).fit()

normal_ols = sm.ols(formula='log_Net_Income ~ log_GDP_per_capita + log_Population_total + log_Investment_total_perc_gdp',
                          data=reg_final_df[(reg_final_df.region == 'East Asia & Pacific')]).fit()

print(normal_ols.summary())

                            OLS Regression Results                            
Dep. Variable:         log_Net_Income   R-squared:                       0.343
Model:                            OLS   Adj. R-squared:                  0.272
Method:                 Least Squares   F-statistic:                     4.866
Date:                Tue, 03 Nov 2020   Prob (F-statistic):            0.00756
Time:                        14:44:23   Log-Likelihood:                -37.705
No. Observations:                  32   AIC:                             83.41
Df Residuals:                      28   BIC:                             89.27
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

In [14]:
# Regression 1b

#normal_ols = sm.ols(formula='log_Net_Income ~ log_GDP_per_capita + log_Population_total + log_Investment_total_perc_gdp',
#                          data=reg_df).fit()

normal_ols = sm.ols(formula='log_regional_revenues ~ log_GDP_per_capita + log_Population_total + log_Investment_total_perc_gdp',
                          data=reg_final_df[(reg_final_df.region == 'East Asia & Pacific')]).fit()

print(normal_ols.summary())

                              OLS Regression Results                             
Dep. Variable:     log_regional_revenues   R-squared:                       0.744
Model:                               OLS   Adj. R-squared:                  0.701
Method:                    Least Squares   F-statistic:                     17.40
Date:                   Tue, 03 Nov 2020   Prob (F-statistic):           1.48e-05
Time:                           14:44:23   Log-Likelihood:                 21.001
No. Observations:                     22   AIC:                            -34.00
Df Residuals:                         18   BIC:                            -29.64
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

In [15]:
# Regression 2

#normal_ols = sm.ols(formula='log_Net_Income ~ log_GDP_per_capita + log_Population_total + log_Investment_total_perc_gdp',
#                          data=reg_df).fit()

normal_ols = sm.ols(formula='log_Net_Income ~ log_GDP_per_capita + log_Population_total + log_Investment_total_perc_gdp',
                          data=reg_final_df[(reg_final_df.region == 'South Asia')]).fit()

print(normal_ols.summary())

                            OLS Regression Results                            
Dep. Variable:         log_Net_Income   R-squared:                       0.402
Model:                            OLS   Adj. R-squared:                  0.337
Method:                 Least Squares   F-statistic:                     6.262
Date:                Tue, 03 Nov 2020   Prob (F-statistic):            0.00218
Time:                        14:44:23   Log-Likelihood:                -36.205
No. Observations:                  32   AIC:                             80.41
Df Residuals:                      28   BIC:                             86.27
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

In [16]:
# Regression 2b

#normal_ols = sm.ols(formula='log_Net_Income ~ log_GDP_per_capita + log_Population_total + log_Investment_total_perc_gdp',
#                          data=reg_df).fit()

normal_ols = sm.ols(formula='log_regional_revenues ~ log_GDP_per_capita + log_Population_total + log_Investment_total_perc_gdp',
                          data=reg_final_df[(reg_final_df.region == 'South Asia')]).fit()

print(normal_ols.summary())

                              OLS Regression Results                             
Dep. Variable:     log_regional_revenues   R-squared:                       0.866
Model:                               OLS   Adj. R-squared:                  0.843
Method:                    Least Squares   F-statistic:                     38.70
Date:                   Tue, 03 Nov 2020   Prob (F-statistic):           4.67e-08
Time:                           14:44:23   Log-Likelihood:                 27.049
No. Observations:                     22   AIC:                            -46.10
Df Residuals:                         18   BIC:                            -41.73
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

In [17]:
# Regression 3

#normal_ols = sm.ols(formula='log_Net_Income ~ log_GDP_per_capita + log_Population_total + log_Investment_total_perc_gdp',
#                          data=reg_df).fit()

normal_ols = sm.ols(formula='log_Net_Income ~ log_GDP_per_capita + log_Population_total + log_Investment_total_perc_gdp',
                          data=reg_final_df[(reg_final_df.region == 'Combined Other Five Regions')]).fit()

print(normal_ols.summary())

                            OLS Regression Results                            
Dep. Variable:         log_Net_Income   R-squared:                       0.388
Model:                            OLS   Adj. R-squared:                  0.323
Method:                 Least Squares   F-statistic:                     5.924
Date:                Tue, 03 Nov 2020   Prob (F-statistic):            0.00291
Time:                        14:44:23   Log-Likelihood:                -36.555
No. Observations:                  32   AIC:                             81.11
Df Residuals:                      28   BIC:                             86.97
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

In [18]:
# Regression 3b

#normal_ols = sm.ols(formula='log_Net_Income ~ log_GDP_per_capita + log_Population_total + log_Investment_total_perc_gdp',
#                          data=reg_df).fit()

normal_ols = sm.ols(formula='log_regional_revenues ~ log_GDP_per_capita + log_Population_total + log_Investment_total_perc_gdp',
                          data=reg_final_df[(reg_final_df.region == 'Combined Other Five Regions')]).fit()

print(normal_ols.summary())

                              OLS Regression Results                             
Dep. Variable:     log_regional_revenues   R-squared:                       0.750
Model:                               OLS   Adj. R-squared:                  0.708
Method:                    Least Squares   F-statistic:                     17.97
Date:                   Tue, 03 Nov 2020   Prob (F-statistic):           1.20e-05
Time:                           14:44:23   Log-Likelihood:                 22.268
No. Observations:                     22   AIC:                            -36.54
Df Residuals:                         18   BIC:                            -32.17
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

In [19]:
## Reg with regional_revenue
#
#normal_ols = sm.ols(formula='log_regional_revenue ~ log_GDP_per_capita + log_Population_total + log_Investment_total_perc_gdp',
#                          data=reg_df).fit() #+ five_other_regions + east_asia + south_asia
#
#print(normal_ols.summary())

*FE by Region*

In [20]:
## FE
#
#FE_ols = sm.ols(sm.ols(formula='log_Net_Income ~ log_GDP_per_capita + log_Population_total + log_Investment_total_perc_gdp + C(region)-1',
#                          data=reg_df).fit())
#print(FE_ols.summary())